## Import des librairies

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from pickle import dump 
import lightgbm as lgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Chargement des données

In [ ]:
data = pd.read_csv("../data/processed/data_preprocessed.csv")
data_with_encoded_cat_columns = pd.read_csv("../data/processed/data_with_encoded_categories_preprocessed.csv")

In [ ]:
data.head();

In [ ]:
data_with_encoded_cat_columns.head();

In [ ]:
data.dtypes;

In [ ]:
def preprocess(data):
    data[["p1_id", "p2_id"]] = data[["p1_id", "p2_id"]].astype('category')
    object_columns = data.select_dtypes(['object']).columns
    data[object_columns] = data[object_columns].astype('category')
    return data

In [ ]:
data = preprocess(data)
data_with_encoded_cat_columns = preprocess(data_with_encoded_cat_columns)

In [ ]:
X = data.drop(columns=['p1_won'])
y = data['p1_won']

In [ ]:
categorical_columns = X.select_dtypes(['category']).columns

In [ ]:
categorical_columns

In [ ]:
categorical_indexes = []
for element in categorical_columns:
    categorical_indexes.append(X.columns.to_list().index(element))

In [ ]:
X_lgb = data_with_encoded_cat_columns.drop(columns=['p1_won'])
y_lgb = data_with_encoded_cat_columns['p1_won']

## Entraînement et Optimisation des modèles

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train_lgb, X_test_lgb, y_train_lgb, y_test_lgb = train_test_split(X_lgb, y_lgb, test_size=0.2, random_state=0)

In [ ]:
grid = {'max_depth': [3,4,5],
        'iterations': [500, 1000, 1500, 2000],
        'learning_rate': [0.01, 0.005, 0.1, 0.05]}
catboost_model = CatBoostClassifier(random_seed=0, cat_features=categorical_indexes, eval_metric="Precision")

In [ ]:
gscv_cat = GridSearchCV(estimator=catboost_model, param_grid=grid, scoring='precision', cv=5)

In [ ]:
gscv_cat.fit(X_train, y_train)

In [ ]:
gscv_cat.best_params_

In [ ]:
gscv_cat.best_score_

In [ ]:
grid_lgb = {'max_depth': [3,4,5],
        'n_estimators': [500, 1000, 1500, 2000],
        'learning_rate': [0.01, 0.005, 0.1, 0.05]}
lgb_model = lgb.LGBMClassifier(categorical_features=categorical_indexes, random_seed=0)

In [ ]:
gscv_lgb = GridSearchCV(estimator=lgb_model, param_grid=grid_lgb, scoring='precision', cv=5)

In [ ]:
gscv_lgb.fit(X_train_lgb, y_train_lgb)

In [ ]:
gscv_lgb.best_params_

In [ ]:
gscv_lgb.best_score_

## Evaluation, Comparaison et Validation des modèles

In [ ]:
catboost_model = CatBoostClassifier(random_seed=0, 
                                    cat_features=categorical_indexes, 
                                    eval_metric="Precision",
                                    iterations=1500,
                                    learning_rate=0.05,
                                    max_depth=4)

In [ ]:
catboost_model.fit(X_train, y_train);

In [ ]:
y_pred_catboost = catboost_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_catboost))

In [ ]:
lgb_model = lgb.LGBMClassifier(random_seed=0, 
                               categorical_features=categorical_indexes,
                               learning_rate=0.005, 
                               max_depth=3,
                               n_estimators=2000)

In [ ]:
lgb_model.fit(X_train_lgb, y_train_lgb)

In [ ]:
y_pred_lgb = lgb_model.predict(X_test_lgb)

In [ ]:
print(classification_report(y_test_lgb, y_pred_lgb))

In [ ]:
catboost_model.save_model('../models/catboost_model.pkl')

In [ ]:
lgb_model.booster_.save_model('../models/lgbm_model.pkl')

In [ ]:
pd.DataFrame({'columns': X.columns, 'feature_importance': catboost_model.feature_importances_}).sort_values(by="feature_importance", ascending=False)

In [ ]:
pd.DataFrame({'columns': X.columns, 'feature_importance': lgb_model.feature_importances_}).sort_values(by="feature_importance", ascending=False)